In [1]:
import numpy as np
import sys
sys.path.append("/cluster/users/tom/git/neon_blink_detection/")

from training.video_loader import video_loader
from src.helper import OfParams
from pathlib import Path
from src.neon_blink_detector import detect_blinks
import typing as T
from helper import OfParams, PPParams
from post_processing import post_process
from xgboost import XGBClassifier
from pathlib import Path
import joblib

video_path = Path("/cluster/users/tom/experiments/neon_blink_detection/datasets/train_data")
of_path = Path("/cluster/users/tom/experiments/neon_blink_detection/datasets/train_data/optical_flow")

of_params = OfParams()

clip_names = np.array(
    [

        "2023-02-01_11-45-11-7621531e",
        "2023-01-27_16-15-26-57802f75",
        "2023-01-27_16-24-04-eb4305b1",
        "2023-01-27_16-10-14-a2a8cbe1"
    ]
)


def get_params() -> T.Tuple[OfParams, PPParams]:
    """Get optical flow parameters and post processing parameters."""

    of_params = OfParams(5, 7, False, (64, 64), 4, 7, 15, 3)
    pp_params = PPParams(
        max_gap_duration_s=0.03,
        short_event_min_len_s=0.1,
        smooth_window=11,
        proba_onset_threshold=0.25,
        proba_offset_threshold=0.25,
    )
    return of_params, pp_params

def get_classifier(clf_path: Path) -> XGBClassifier:
    """Get classifier with weights."""

    return joblib.load(str(clf_path))



/cluster/anaconda3/envs/tom_py310/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
from features_calculator import calculate_optical_flow, concatenate_features

of_params, pp_params = get_params()

clf = get_classifier("/cluster/users/tom/git/neon_blink_detection/weights/xgb.sav")

rec = video_loader(of_params)
ts, images_left, images_right = rec._get_frames(clip_names[2], convert_to_gray=True)
blink_df = rec._load_gt_labels(clip_names[2])
feature_array, _ = rec._compute_optical_flow(of_params, images_left, images_right)
features = concatenate_features(feature_array, of_params)
proba = clf.predict_proba(features)
blink_events = post_process(ts, proba, pp_params)


In [87]:
from src.utils import resize_images

left_images, right_images = resize_images(
    images_left, images_right, img_shape=(64, 64)
)

blink_ts = blink_df[blink_df["label"]=="onset"]["start_ts"]
blink_on_idx = np.where(np.isin(ts, blink_ts))[0]-20
blink_ts = blink_df[blink_df["label"]=="offset"]["end_ts"]
blink_off_idx = np.where(np.isin(ts, blink_ts))[0]+20


img_left = np.concatenate([left_images[blink_on_idx[x]: blink_off_idx[x], :, :] for x in range(len(blink_on_idx))])
img_right = np.concatenate([right_images[blink_on_idx[x]: blink_off_idx[x], :, :] for x in range(len(blink_on_idx))])

predicted_blink_on = np.array([np.where(np.isin(ts, blink_events[x].start_time))[0][0] -20 for x in range(len(blink_events))])

predicted_blink_off = np.array([np.where(np.isin(ts, blink_events[x].end_time))[0][0]-20 for x in range(len(blink_events))])


In [153]:


# img_left = np.concatenate([left_images[blink_on_idx[x]: blink_off_idx[x], :, :] for x in range(len(blink_on_idx))])
# img_right = np.concatenate([right_images[blink_on_idx[x]: blink_off_idx[x], :, :] for x in range(len(blink_on_idx))])

img_left = left_images[predicted_blink_on[72]:predicted_blink_off[72], :, :]



In [154]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# np array with shape (frames, height, width, channels)
video = img_left

fig, axs = plt.subplots(1,2)
im0 = axs[0].imshow(video[0,:,:], cmap='gray')
axs[0].axis("off")

# im1 = axs[1].imshow(zz[:,:,0], vmin=-10, vmax=10)
# axs[1].axis("off")

plt.close() # this is required to not display the generated image

def init():
    im0.set_data(video[0,:,:])
    # im1.set_data(zz[:,:,0])

def animate(i):
    im0.set_data(video[i,:,:])
    # im1.set_data(zz[:,:,i])
    return im0#, im1

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
                               interval=25)
HTML(anim.to_html5_video())

In [127]:
start_of_overlap = np.maximum(
    blink_on_idx[:, np.newaxis],
    predicted_blink_on[np.newaxis, :],
)
end_of_overlap = np.minimum(
    blink_off_idx[:, np.newaxis],
    predicted_blink_off[np.newaxis, :],
)
overlap = end_of_overlap - start_of_overlap
overlap[overlap <= 0] = 0

max_end_times = np.maximum(
    blink_off_idx[:, np.newaxis],
    predicted_blink_off[np.newaxis, :],
)
min_start_times = np.minimum(
    blink_on_idx[:, np.newaxis],
    predicted_blink_on[np.newaxis, :],
)
union = max_end_times - min_start_times

iou = overlap / union


In [132]:
iou_thr = 0.2
over_thr = iou * (iou > iou_thr)
over_thr[over_thr == 0] = -1
found_match = np.any(over_thr > 0, axis=1)
# if there is a match, indicates the index of the match
ind_match = np.nanargmax(over_thr, axis=1)
# indicates match index, or (-1) if not match is found
ind_match[~found_match] = -1

In [3]:
blink_ts = blink_df[blink_df["label"]=="onset"]["start_ts"]
blink_on_idx = np.where(np.isin(ts, blink_ts))[0]
blink_ts = blink_df[blink_df["label"]=="offset"]["end_ts"]
blink_off_idx = np.where(np.isin(ts, blink_ts))[0]

predicted_blink_on = np.array([np.where(np.isin(ts, blink_events[x].start_time))[0][0] for x in range(len(blink_events))])
predicted_blink_off = np.array([np.where(np.isin(ts, blink_events[x].end_time))[0][0] for x in range(len(blink_events))])

In [4]:
from matplotlib.patches import Rectangle

def render_event_array(ax, blink_on_idx, blink_off_idx, y, color):
    # flags = np.ones(len(legend_labels))
    legends = {}

    for i in range(len(blink_on_idx)):
        start = blink_on_idx[i]/200
        end = blink_off_idx[i]/200
        height = 0.5
        patch = Rectangle((start, y), end - start, height, color=color)
        ax.add_patch(patch)
        # legends["blink"] = patch

    ax.set_yticks([])
    ax.set_ylim(0, 1.5)



In [ ]:
f, ax = plt.subplots(6, 1)

f.set_size_inches(20, 20)
render_event_array(ax[0], blink_on_idx, blink_off_idx, 0.2, color=[0.2, 0.8, 0.4])
render_event_array(ax[0], predicted_blink_on, predicted_blink_off, 0.8, color=[1, 0.4, 0.2])
ax[0].set_xlim(0, 30)

render_event_array(ax[1], blink_on_idx, blink_off_idx, 0.2, color=[0.2, 0.8, 0.4])
render_event_array(ax[1], predicted_blink_on, predicted_blink_off, 0.8, color=[1, 0.4, 0.2])
ax[1].set_xlim(30, 60)

render_event_array(ax[2], blink_on_idx, blink_off_idx, 0.2, color=[0.2, 0.8, 0.4])
render_event_array(ax[2], predicted_blink_on, predicted_blink_off, 0.8, color=[1, 0.4, 0.2])
ax[2].set_xlim(60, 90)

render_event_array(ax[3], blink_on_idx, blink_off_idx, 0.2, color=[0.2, 0.8, 0.4])
render_event_array(ax[3], predicted_blink_on, predicted_blink_off, 0.8, color=[1, 0.4, 0.2])
ax[3].set_xlim(90, 120)

render_event_array(ax[4], blink_on_idx, blink_off_idx, 0.2, color=[0.2, 0.8, 0.4])
render_event_array(ax[4], predicted_blink_on, predicted_blink_off, 0.8, color=[1, 0.4, 0.2])
ax[4].set_xlim(120, 150)

render_event_array(ax[5], blink_on_idx, blink_off_idx, 0.2, color=[0.2, 0.8, 0.4])
render_event_array(ax[5], predicted_blink_on, predicted_blink_off, 0.8, color=[1, 0.4, 0.2])
ax[5].set_xlim(150, 180)

# render_event_array(ax[5], blink_on_idx, blink_off_idx, 0.2, color=[0.2, 0.8, 0.4])
# render_event_array(ax[5], predicted_blink_on, predicted_blink_off, 0.8, color=[1, 0.4, 0.2])
# ax[5].set_xlim(180, 210)
ax[5].set_xlabel("Time (s)")

In [6]:
import matplotlib.pyplot as plt

In [327]:
predicted_blink_on

array([   50,   414,   650,   747,   981,  1072,  1142,  1380,  1643,
        1970,  2168,  2256,  2583,  2874,  3681,  4094,  4462,  4534,
        5996,  6123,  6261,  7688,  8018,  9751,  9833, 13114, 13190,
       14846, 15081, 15393, 15685, 16548, 17336, 17442, 18734, 20180,
       20601, 21346, 21809, 22044, 22697, 22763, 23602, 23965, 24351,
       25891, 26239, 26630, 26699, 29236, 29350, 29400, 29459, 29600,
       29687, 29793, 29901, 30042, 30330, 30379, 30826, 31703, 31803,
       32588, 32692, 33523, 33873, 34921, 35088, 35774, 36488, 37835,
       38686, 39255, 40014, 40412, 41407, 41572, 42789, 43167, 43399,
       43908, 44159, 44679, 44864, 45167, 45548, 45745, 46046, 46117,
       46144, 46348, 46620, 46679])